# Step 19 Improvement in Variable

## 19.1 Setting the Variable name

In [1]:

import numpy as np
import weakref
import contextlib

class Config:
    enable_backprop=True

@contextlib.contextmanager 
    
def using_config(name, value):
    old_value=getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)    

def no_grad():
    return using_config('enable_backprop', False)
        
class Variable:
    def __init__(self, data, name=None): #added name=None
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.name=name #added
        self.data = data
        self.grad = None
        self.creator = None
        self.generation=0

    def set_creator(self, func):
        self.creator = func
        self.generation= func.generation+1 

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set=set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs=(gxs, )
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad=gx
                else:
                    x.grad=x.grad+gx 
            
                if x.creator is not None:
                    add_func(x.creator)
            
            if not retain_grad:
                for y in f.outputs:
                    y().grad=None
    def cleargrad(self):
        self.grad=None

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [2]:
x=Variable(np.array(1.0), 'input_x')

Now, the name of the variable x is __input_x__.

## 19.2 ndarray Variable

__shape__ tells you the 'shape' of the multi-dimensional array. 

In [5]:
import numpy as np

x=np.array([[1,2,3],[4,5,6]])
x.shape

(2, 3)

In [6]:
class Variable:
    def __init__(self, data, name=None): #added name=None
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.name=name #added
        self.data = data
        self.grad = None
        self.creator = None
        self.generation=0

    def set_creator(self, func):
        self.creator = func
        self.generation= func.generation+1 

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set=set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs=(gxs, )
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad=gx
                else:
                    x.grad=x.grad+gx 
            
                if x.creator is not None:
                    add_func(x.creator)
            
            if not retain_grad:
                for y in f.outputs:
                    y().grad=None
    def cleargrad(self):
        self.grad=None
    
    @property
    def shape(self):
        return self.data.shape

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [7]:

x=Variable(np.array([[1,2,3],[4,5,6]]))
print(x.shape)

(2, 3)


In [9]:
class Variable:
    def __init__(self, data, name=None): #added name=None
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.name=name #added
        self.data = data
        self.grad = None
        self.creator = None
        self.generation=0

    def set_creator(self, func):
        self.creator = func
        self.generation= func.generation+1 

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set=set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs=(gxs, )
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad=gx
                else:
                    x.grad=x.grad+gx 
            
                if x.creator is not None:
                    add_func(x.creator)
            
            if not retain_grad:
                for y in f.outputs:
                    y().grad=None
    def cleargrad(self):
        self.grad=None
    
    @property
    def ndim(self):
        return self.data.ndim
    
    @property
    def shape(self):
        return self.data.shape
    
    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

## 19.3 len and print function

__len__ function returns the number of elements included inside the array, list etc.

In [10]:
class Variable:
    def __init__(self, data, name=None): #added name=None
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.name=name #added
        self.data = data
        self.grad = None
        self.creator = None
        self.generation=0

    def set_creator(self, func):
        self.creator = func
        self.generation= func.generation+1 

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set=set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs=(gxs, )
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad=gx
                else:
                    x.grad=x.grad+gx 
            
                if x.creator is not None:
                    add_func(x.creator)
            
            if not retain_grad:
                for y in f.outputs:
                    y().grad=None
    def cleargrad(self):
        self.grad=None
    
    @property
    def ndim(self):
        return self.data.ndim
    
    @property
    def shape(self):
        return self.data.shape
    
    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype
    
    def __len__(self): #now we can count elements inside the Variable class.
        return len(self.data)

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [14]:
x=Variable(np.array([[1,2,3],[4,5,6]]))
print(len(x))

2


For the last, we add __repr__ in order to print out the data inside Variable.

In [18]:
class Variable:
    def __init__(self, data, name=None): #added name=None
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.name=name #added
        self.data = data
        self.grad = None
        self.creator = None
        self.generation=0

    def set_creator(self, func):
        self.creator = func
        self.generation= func.generation+1 

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set=set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs=(gxs, )
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad=gx
                else:
                    x.grad=x.grad+gx 
            
                if x.creator is not None:
                    add_func(x.creator)
            
            if not retain_grad:
                for y in f.outputs:
                    y().grad=None
    def cleargrad(self):
        self.grad=None
    
    @property
    def ndim(self):
        return self.data.ndim
    
    @property
    def shape(self):
        return self.data.shape
    
    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype
    
    def __len__(self): 
        return len(self.data)
    
    def __repr__(self): #print out the data inside Variable
        if self.data is None:
            return 'variable(None)'
        p=str(self.data).replace('\n','\n'+''*9)
        return 'variable('+p+')'

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [17]:
print(Variable(np.array([[1,2,3],[4,5,6]])))

variable([[1 2 3]
 [4 5 6]])
